In [6]:
import json


In [7]:
# load data/cihly/parametry.json
with open('data/bricks/parameters.json') as f:
    data = json.load(f)
# print data
print(data)
dx = data['dx']
dy = data['dy']
dt = data['dt']
print(f"dx = {dx}, dy = {dy}, dt = {dt}")

{'dx': 0.005, 'dy': 0.005, 'dt': 2.0}
dx = 0.005, dy = 0.005, dt = 2.0


In [4]:
# load data/spirala/parametry.json
with open('data/spiral/parameters.json') as f:
    data = json.load(f)
# print data
print(data)
dx = data['dx']
dy = data['dy']
dt = data['dt']
print(f"dx = {dx}, dy = {dy}, dt = {dt}")

{'dx': 0.01, 'dy': 0.01, 'dt': 0.1}
dx = 0.01, dy = 0.01, dt = 0.1
